In [5]:
import pylab as p
import numpy as np
from itertools import product

def max_dot_monomials(a,b):
    c = p.empty_like(a)
    A = range(a.ndim)
    for i,j in product(A, A):
        c[i,j] = p.array( a[i,:]).flatten() + p.array( b[:,j]).flatten()
    return c


def maxdot(a,b):
    """currently only square matrices a,b are supported"""
    c = p.empty_like(a)
    A = range(a.ndim)
    for i,j in product(A, A):
        c[i,j] = max(p.array( a[i,:]).flatten() + p.array( b[:,j]).flatten())
    return c

def mindot(a,b):
    c = p.empty_like(a)
    A = range(a.ndim)
    for i,j in product(A, A):
        c[i,j] = min(p.array( a[i,:]).flatten() + p.array( b[:,j]).flatten())
    return c

def test():
    x,y,z,w = 1,2,3,4
    a = p.matrix([[x,y],[z,w]])
    return mindot(a,a)

# print test()

In [16]:
def nullstellen(a, ll, ru, eps = 10e-1):
    """
    compute zeros of the partially linear (matrix) function 'a' 
    heuristic simply looks for sufficiently small function values (< eps) and takes them as zero
    :arg a: function a:RR^3 -> RR^n where the image can be a matrix or vector (tested for type 'numpy.array')
    :param ll: lower left coordinates (x_0, y_0)
    :param ru: right upper coordinates (x_1, y_1)
    :param eps: accuracy of the computation
    """
    z_res = p.arange(-1+eps/2,1,eps) # +eps/2 to get also zeros right
    X, Y = p.mgrid[ll[0]:ru[0]:eps, ll[1]:ru[1]:eps]
    Z = []
    Z.append( p.zeros_like(X) )
    #print Z
    for (i, x), (j, y), z in product( enumerate(X[:,0]), enumerate(Y[0,:]), z_res):
        if a(x,y,z).all() < eps:   # all to get intersection of all equations
            all_occupied = False
            for k in range(len(Z)):
                if Z[k][i,j] != 0:
                    all_occupied = True
                else: 
                    Z[k][i,j] = z
                    all_occupied = False
                    break
            if all_occupied:
                Z.append( p.zeros_like(X) )
                Z[k+1][i,j] = z
    return X,Y,Z

def make_plot():
    X, Y, Z = nullstellen(b, ll = (-1,-1), ru = (3,3), eps = 5*10e-2)
    fig = p.figure()
    ax = fig.add_subplot(1,1,1, projection='3d')
    print len(Z) # 4 layers
    for k in range(len(Z)):
        surf = ax.plot_surface(X,Y,Z[k])
    p.show()

make_plot()

4


In [6]:
def randrange(n, vmin, vmax):
    return (vmax-vmin)*np.random.rand(n) + vmin

def randiter(n, vmin, vmax):
    for x in (vmax-vmin)*np.random.rand(n) + vmin:
        yield x


def nullstellen(a, ll, ru, eps = 10e-1):
    """
    compute zeros of the partially linear (matrix) function 'a' 
    heuristic simply looks for sufficiently small function values (< eps) and takes them as zero
    :arg a: function a:RR^3 -> RR^n where the image can be a matrix or vector (tested for type 'numpy.array')
    :param ll: lower left coordinates (x_0, y_0)
    :param ru: right upper coordinates (x_1, y_1)
    :param eps: accuracy of the computation
    """
    n = int(1/eps)
    z_res = randiter(n, ll[0],ru[0])
    xs = randrange(n, ll[0],ru[0])
    ys = randrange(n, ll[1],ru[1])
    zs = p.zeros_like(xs)
    #print Z
    for i, x, y in zip(range(n), xs, ys):
        #print i,j
        for z in z_res:
            if a(x,y,z).all() < eps:   # all to get intersection of all equations
                zs[i] = z
                break
    return xs,ys,zs

In [ ]:
a = lambda x,y,z: p.matrix([[x,y],[z,0]])
b = lambda x,y,z: max_dot_monomials(a(x,y,z),a(x,y,z)) - a(x,y,z)


def tropical_hypersurface(b, ll, ru, eps = 10e-1):
    """
    compute points in the (matrix) function 'a' where maximum is attained at least twice
    
    :arg a: function a:RR^3 -> (RR^k)^n matrix or vector of list of tropical monomials
    :param ll: lower left coordinates (x_0, y_0)
    :param ru: right upper coordinates (x_1, y_1)
    :param eps: accuracy of the computation
    """
    z_res = p.arange(-1,1,eps)
    X, Y = p.mgrid[ll[0]:ru[0]:eps, ll[1]:ru[1]:eps]
    Z = p.zeros_like(X)
    #print Z
    for (i, x), (j, y), z in product( enumerate(X[:,0]), enumerate(Y[0,:]), z_res):
        #print i,j
        
        for z in z_res:
            for a in a(x,y,z).flatten():
                max_ = a(x,y,z) # .all()   # maximum is attained at least twice
                if min(z - max_) < eps:
                    Z[i,j] = z

xs, ys, zs = tropical_hypersurface(b, ll = (-1,-1), ru = (5,5), eps = 10e-5)
                    

In [7]:
from mpl_toolkits.mplot3d import Axes3D

a = lambda x,y,z: p.matrix([[x,y],[z,0]])
b = lambda x,y,z: maxdot(a(x,y,z),a(x,y,z)) - a(x,y,z)

#interval = p.linspace(0,1,10)

xs, ys, zs = nullstellen(b, ll = (-1,-1), ru = (5,5), eps = 10e-5)
fig = p.figure()
ax = fig.add_subplot(1,1,1, projection='3d')
ax.scatter(xs, ys, zs)
#ax.set_xlabel('X Label')
#ax.set_ylabel('Y Label')
#ax.set_zlabel('Z Label')

p.show()

In [121]:
np.random.rand(5)

array([ 0.70010497,  0.58377023,  0.16957092,  0.09749866,  0.73008557])

In [20]:
x_res = p.arange( 0,1, 0.1)
y_res = p.arange( 0,1, 0.1)
#z_res = p.arange(-1,1,eps)
grid = p.mgrid[x_res, y_res]

AttributeError: 'numpy.ndarray' object has no attribute 'step'

In [40]:
for f in (maxdot(a,a) - a).flat[:]:
    print f

4
6
2
4


In [71]:
np.array([[1,1],[2,2]]).flat[:]

array([1, 1, 2, 2])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')
surf = ax.plot_surface(nullstellen( b, ll = (-1,-1), ru = (1,1), eps = 10e-1))
plt.show()

In [8]:
import numpy as np
grid = p.mgrid[0:1:0.1, 0:1:0.1]
grid


array([[[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
        [ 0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2],
        [ 0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3],
        [ 0.4,  0.4,  0.4,  0.4,  0.4,  0.4,  0.4,  0.4,  0.4,  0.4],
        [ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5],
        [ 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6],
        [ 0.7,  0.7,  0.7,  0.7,  0.7,  0.7,  0.7,  0.7,  0.7,  0.7],
        [ 0.8,  0.8,  0.8,  0.8,  0.8,  0.8,  0.8,  0.8,  0.8,  0.8],
        [ 0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9]],

       [[ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9],
        [ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9],
        [ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9],
        [ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9],
        [ 0. ,  0.

NameError: name 'X' is not defined